In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
import ast
from sklearn.model_selection import train_test_split
import albumentations as A
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict


warnings.filterwarnings('ignore')



In [2]:
image_dir = r'C:\Users\User\Documents\Фотопримеры для модели ИИ\\'
image_dir_new = r'C:\Users\User\Documents\датасет ВКР\\'

folder_to_classes = {
    'переполненная_площадка_КГО': [3],
    'не_переполненая_площадка_КГО': [4],
    'КГО_переполненое': [14],
    'КГО_не_переполненный': [15]
}

images = []
labels = []



In [3]:
df30=pd.read_csv("df30.csv")


In [4]:
class_counts = defaultdict(int) 

for _, row in df30.iterrows():
    img_path = os.path.join(image_dir, row['file_name'])
    
    try:
        img = image.load_img(img_path, target_size=(224, 224))  
        img_array = image.img_to_array(img)

        parsed_labels = ast.literal_eval(str(row['OUTPUT:classes']))
        
        if isinstance(parsed_labels, list):
            if 16 in parsed_labels:  
                continue
            
            labels.append(parsed_labels)
            images.append(img_array)
            
            for label in parsed_labels:
                class_counts[label] += 1

    except (FileNotFoundError, OSError) as e:
        continue


In [6]:
###  1.1 Проверка распределения классов после первой папки
print("\nРаспределение классов после первой папки (Фотопримеры для модели ИИ):")
for cls, count in sorted(class_counts.items()):
    print(f"Класс {cls}: {count} изображений")




Распределение классов после первой папки (Фотопримеры для модели ИИ):
Класс 1: 1837 изображений
Класс 2: 7831 изображений
Класс 3: 789 изображений
Класс 4: 710 изображений
Класс 5: 10240 изображений
Класс 6: 635 изображений
Класс 7: 3504 изображений
Класс 8: 4267 изображений
Класс 9: 1520 изображений
Класс 10: 5954 изображений
Класс 11: 1889 изображений
Класс 13: 796 изображений
Класс 14: 93 изображений
Класс 15: 258 изображений


In [7]:
additional_folder_path = r'C:\Users\User\Documents\ещёконтейнеры\\12 класс'
if os.path.exists(additional_folder_path):
    for file_name in os.listdir(additional_folder_path):
        img_path = os.path.join(additional_folder_path, file_name)
        
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append([12])  # Явно указываем класс 12
            
            # Обновляем статистику
            class_counts[12] += 1

        except (FileNotFoundError, OSError) as e:
            print(f"Ошибка при обработке {img_path}: {e}")
            continue
else:
    print(f"\Папка с дополнительными контейнерами не найдена: {additional_folder_path}")


In [8]:
additional_folder_path = r'C:\Users\User\Documents\ещёконтейнеры\\14 класс'
if os.path.exists(additional_folder_path):
    for file_name in os.listdir(additional_folder_path):
        img_path = os.path.join(additional_folder_path, file_name)
        
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append([12])  # Явно указываем класс 12
            
            # Обновляем статистику
            class_counts[12] += 1

        except (FileNotFoundError, OSError) as e:
            print(f"Ошибка при обработке {img_path}: {e}")
            continue
else:
    print(f"\Папка с дополнительными контейнерами не найдена: {additional_folder_path}")


In [9]:
for folder, class_labels in folder_to_classes.items():
    folder_path = os.path.join(image_dir_new, folder)
    
    if not os.path.exists(folder_path):
        print(f"Папка не найдена: {folder_path}, пропускаем.")
        continue

    for file_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file_name)
        
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append(class_labels)  # Привязываем классы из названия папки
            # Обновляем статистику классов
            for label in class_labels:
                class_counts[label] += 1

        except (FileNotFoundError, OSError) as e:
            print(f"Ошибка при обработке {img_path}: {e}")
            continue


In [10]:
print("\nРаспределение классов:")
for cls, count in sorted(class_counts.items()):
    print(f"Класс {cls}: {count} изображений")



Распределение классов:
Класс 1: 1837 изображений
Класс 2: 7831 изображений
Класс 3: 1050 изображений
Класс 4: 896 изображений
Класс 5: 10240 изображений
Класс 6: 635 изображений
Класс 7: 3504 изображений
Класс 8: 4267 изображений
Класс 9: 1520 изображений
Класс 10: 5954 изображений
Класс 11: 1889 изображений
Класс 12: 273 изображений
Класс 13: 796 изображений
Класс 14: 275 изображений
Класс 15: 2273 изображений


In [11]:
images = np.array(images)
labels = np.array(labels, dtype=object)



In [12]:
images = np.array(images) / 255.0  
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)  


In [13]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)


In [14]:
print("Распределение в обучающей выборке:")
print(np.sum(y_train, axis=0))
print("\nРаспределение в тестовой выборке:")
print(np.sum(y_test, axis=0))

Распределение в обучающей выборке:
[1267 5465  732  623 7141  444 2444 2952 1068 4121 1331  205  560  193
 1606]

Распределение в тестовой выборке:
[ 570 2366  318  273 3099  191 1060 1315  452 1833  558   68  236   82
  667]


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import numpy as np

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

augmented_images = []
augmented_labels = []

augment_per_image = 1  

for i in range(len(X_train)):
    img = X_train[i]
    label = y_train[i]

    img = img.reshape((1,) + img.shape)  

    aug_iter = datagen.flow(img, batch_size=1)

    for _ in range(augment_per_image):
        aug_img = next(aug_iter)[0]  
        augmented_images.append(aug_img)
        augmented_labels.append(label)

X_train = np.concatenate([X_train, np.array(augmented_images)])
y_train = np.concatenate([y_train, np.array(augmented_labels)])

print(f"Аугментация завершена. Новое количество изображений в трейне: {len(X_train)}")


Аугментация завершена. Новое количество изображений в трейне: 21786


In [17]:
np.save('images_train.npy', X_train)
np.save('images_test.npy', X_test)
np.save('labels_train.npy', y_train)
np.save('labels_test.npy', y_test)